In [ ]:
import numpy as np
from numpy import pi
from scipy.interpolate import CubicSpline
from integrate.lowStorageRungeKuttaMethods import LSRK3
from derivative.spectralMethods import spectralMethod1
from utils.grid import wallNormalGrid
from scipy.sparse.linalg import LinearOperator, gmres
from utils.wallLaw import velocityProfile

In [ ]:
class channel2(object):
    nx = 384
    ny = 255
    lx = 2. * pi
    Ret = 400
    nu = 1.
    delta = 1.
    eta = 0.97
    dt = 0.01
    def __init__(self):
        self.x = np.linspace(0, self.lx, self.nx, endpoint=False)
        self.y = wallNormalGrid(self.ny, self.eta, self.delta)
        self.dx = self.lx / self.nx
        self.integrate = LSRK3()
        self.spectral = spectralMethod1(n=self.nx, d=self.dx)
        self.force = velocityProfile(self.Ret) ** -2
        self.p_shape = (self.ny, self.nx)
        self.A_shape = (self.nx*self.ny, self.nx*self.ny)
        self.exitCode = 0
        self.A = LinearOperator(self.A_shape, matvec=self.lhs2)

    def rhs1(self, t, u):
        u_interp = CubicSpline(self.y, u, axis=-2, bc_type=((0, 0.), (0, 0.)))
        return ( - u[0] * self.spectral.diff_phys(u)
                 - u[1] * u_interp(self.y, nu=1)
                 + self.nu * (self.spectral.diff_phys(u, order=2)
                            + u_interp(self.y, nu=2))
                 + self.force )

    def lhs2(self, p_flatten):
        p = np.reshape(p_flatten, self.p_shape)
        p_interp = CubicSpline(self.y, p, axis=-2, bc_type=((1, 0.), (1, 0.)))
        p = self.spectral.diff_phys(p, order=2) + p_interp(self.y, nu=2)
        return p.flatten()

    def rhs2(self, u):
        u_interp = CubicSpline(self.y, u[1], axis=-2, bc_type=((0, 0.), (0, 0.)))
        return (self.spectral.diff_phys(u[0])
                + u_interp(self.y, nu=1)) / self.dt

    def rhs3(self):
        p_interp = CubicSpline(self.y, self.p, axis=-2, bc_type=((1, 0.), (1, 0.)))
        return np.stack((self.spectral.diff_phys(self.p, order=1), p_interp(self.y, nu=1)))

    def step(self):
        u = self.integrate.step(fun=self.rhs1, u=self.u, t=0, dt=self.dt)
        p_flatten, self.exitCode = gmres(A=self.A, b=self.rhs2(u), x0=self.p)
        self.p = np.reshape(p_flatten, self.p_shape)
        self.u -= self.dt * self.rhs3()

    def solve(self, u0, t0, ts_span, p0=None):
        self.u = u0
        self.t = t0
        self.p = p0
        for ts in range(ts_span[0], ts_span[1]+1):
            self.step()
            self.t = t0 + self.dt*ts

In [ ]:
tcf = channel2()
u0 = np.zeros((tcf.ny, tcf.nx))
v0 = np.zeros((tcf.ny, tcf.nx))
# u0[:,None] = velocityProfile(np.abs(tcf.y))
U0 = np.stack((u0, v0))
t0 = 0
ts_span = (0, 10)
# tcf.solve(U0, t0, ts_span)